# gameplay

> Fill in a module description here

In [ ]:
#| default_exp gameplay

In [1]:
#| hide
from nbdev.showdoc import *

In [2]:
%load_ext autoreload


In [3]:
%autoreload 2

In [4]:
#| export
import clue.model as model
import clue.ui as ui
from random import choice,shuffle,choices
import os

In [5]:
#| hide
stgame = model.createClue(4)
stmoves = []

In [6]:
#| export
def getPlay(pc,game, moves):
    
    

    ui.showBoard(pc,game,moves)
    playerRoom = model.roomOf(pc,game)
    passages = model.roomsAround(playerRoom)
    locs = [[3,x] for x in passages ]
    print("Move to Room:\n")
 
    l = ui.chooseKey(locs,pc,game,moves)
    l = passages[l-1]

    wKeys = [[1,x] for x in range(len(model.people))]
    p = ui.chooseKey(wKeys,1,game,moves) - 1

    wKeys = [[2,x] for x in range(len(model.weapons))]
    w = ui.chooseKey(wKeys,1,game,moves) - 1

    return p, w, l



In [7]:
#| hide
stguess = getPlay(1,stgame,stmoves)
print(ui.showChoice(stguess))
def seenCounts(pc,game, moves):
    knoweledge = getSeen(pc,game, moves)
    p = [0,0,0]
    for x in knoweledge:
        i = int(x/100) -1
        p[i] += 1

    return p

╒══════════════╤══════════════════╤══════════════╕
│ 2) Ball Room │ 3) Billiard Room │ Conservatory │
│    S         │    P             │    P         │
├──────────────┼──────────────────┼──────────────┤
│ 1) Study     │                  │ Dining Room  │
│    G         │                  │              │
├──────────────┼──────────────────┼──────────────┤
│ Library      │                  │ Hall         │
│    W         │                  │              │
├──────────────┼──────────────────┼──────────────┤
│              │ Lounge           │ Kitchen      │
│              │                  │    M         │
╘══════════════╧══════════════════╧══════════════╛
Move to Room:

1 Study
2 Ball Room
3*Billiard Room


Choose from 1 to 3:  2


1 👠 Miss Scarlet
2 🧐 Colonel Mustard
3*♡  Mrs. White
4*🐉 Mr. Green
5 🍆 Professor Plum
6 🦚 Mrs. Peacock


Choose from 1 to 6:  3


1 🔪 Knife
2*🔫 Revolver
3 🪢 Rope
4 🔧 Wrench
5 🕯️ Candlestick
6 🪠 Lead Pipe


Choose from 1 to 6:  1


♡  Mrs. White with the 🔪 Knife in the Ball Room


In [8]:
#| export




def simpleGuess(pc,game, moves):


    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    pdeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  pdeck if x not in exclude]
    p = choice(pdeck)%100
    
    pdeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in pdeck if x not in exclude]
    w = choice(pdeck)%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)


In [105]:
#| export

def itemsForType(t):
    if t == 1:
        return model.people
    if t == 2:
        return model.weapons
    return model.rooms
        
def itemGuess(t,moves):
    l = itemsForType(t)

    ret = [1 for x in range(len(l))]
    for x in moves:
        ret[x[t-1]] += 1
    #print(ret)
    return ret
    
def showCounts(t,moves):
    items = itemGuess(t,moves)
    base = itemsForType(t)
    #print(items,base)
    for i in range(len(items)):
        print(i,ui.cardFull(base[i]),items[i])
    

def betterGuess(pc,game, moves):
    memory = 3

    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    examine = moves[-memory:]
    #examine = []
    
    keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  keydeck if x not in exclude]
    pC = itemGuess(1,examine)
    pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]
    
    p = choices(pdeck,weights=pWeights)
    p = p[0]%100
    
    keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in keydeck if x not in exclude]
    pC = itemGuess(2,examine)
    pWeights = [pC[x%100] * pC[x%100] for x in  keydeck if x not in exclude]
    
    w = choices(pdeck,weights=pWeights)[0]%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)

def deepGuess(pc,game, moves):
    memory = 20

    playerRoom = model.roomOf(pc,game)
    exclude  = set(model.getSeen(pc,game, moves))
    
    keydeck = [model.keyCard(1,x) for x in range(len(model.people))]
    pdeck = [x for x in  keydeck if x not in exclude]
    pC = itemGuess(1,moves[-memory:])
    pWeights = [pC[x%100] for x in  keydeck if x not in exclude]
    p = choices(pdeck,weights=pWeights)
    p = p[0]%100
    
    keydeck = [model.keyCard(2,x) for x in range(len(model.weapons))]
    pdeck = [x for x in keydeck if x not in exclude]
    pC = itemGuess(2,moves[-memory:])
    pWeights = [pC[x%100] for x in  keydeck if x not in exclude]
    w = choices(pdeck,weights=pWeights)[0]%100
    
    pdeck = [model.keyCard(3,x) for x in model.roomsAround(playerRoom)]
    rdeck = [x for x in  pdeck if x not in exclude]
    
    if len(rdeck) == 0:
        l = pdeck[0]%100
    else:
        l = choice(rdeck)%100
    
    return (p,w,l)

[1, 2, 3, 4, 5]

## Visual

In [89]:
#| export
def playerName(i):
    return ui.cardFull(model.cardLookup(1,i-1))
    
def runVisualGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
    lastM = ["" for x in range(numPlayers)]
   
             
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        if isUser:
            ui.printPlayerHand(player,agame)
            #showCounts(1,amoves)
            #showCounts(2,amoves)
            print(lastM[player-1])
        
        ans = strat(player,agame,amoves)
        if isUser:
            pass
            #os.system("clear")
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        print( playerName(player) + " suspects " + ui.showChoice(ans)+".")
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    print("\n\n gave a false one " +  ui.cardFull(model.keyLookUp(x))+"!")
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)
            if isUser:
                msg = playerName(check[-2]) + " has " + ui.cardFull(model.keyLookUp(card))+"!"
            else:
                msg = playerName(check[-2]) + " responds."
            print(msg)
            lastM[player-1] = msg
            
            player = player + 1
            if player - 1 >=len(strats):
               player = 1 
    
    print("\n" + playerName(player) + " won!")
    ui.printGame(agame) 

In [93]:
#| hide
s1 = ("simple",False,simpleGuess)
b1 = ("better",False,betterGuess)
d1 =("deep",False,deepGuess)
strategies = [ s1,b1,s1]
runVisualGame(strategies)

👠 Miss Scarlet suspects ♡  Mrs. White with the 🔧 Wrench in the Conservatory.
♡  Mrs. White responds.
[1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
🧐 Colonel Mustard suspects 🐉 Mr. Green with the 🪠 Lead Pipe in the Kitchen.
👠 Miss Scarlet responds.
♡  Mrs. White suspects 🐉 Mr. Green with the 🕯️ Candlestick in the Billiard Room.
🧐 Colonel Mustard responds.
👠 Miss Scarlet suspects 👠 Miss Scarlet with the 🔧 Wrench in the Billiard Room.
🧐 Colonel Mustard responds.
[1, 1, 1, 1, 1] [1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
🧐 Colonel Mustard suspects 🍆 Professor Plum with the 🔫 Revolver in the Hall.
♡  Mrs. White responds.
♡  Mrs. White suspects 🐉 Mr. Green with the 🪠 Lead Pipe in the Conservatory.
👠 Miss Scarlet responds.
👠 Miss Scarlet suspects 🍆 Professor Plum with the 🔧 Wrench in the Conservatory.
♡  Mrs. White responds.
[1, 1, 1, 1] [1, 1, 1, 1, 1, 1]
[1, 1, 1, 1]
🧐 Colonel Mustard suspects 🐉 Mr. Green with the 🪠 Lead Pipe in the Dining Room.
👠 Miss Scarlet responds.
♡  Mrs. White suspects 🐉 Mr. 

In [92]:
def runGame( strats):
    numPlayers = len(strats)
    done = False
    agame = model.createClue(numPlayers)
    amoves = []
    player = 1
          
    while not done:
        #print(seenCounts(player,agame,amoves))
        
        stratName, isUser, strat = strats[player - 1]
        
        ans = strat(player,agame,amoves)
        agame = model.relocatePlayers(player , ans[0],ans[2],agame)
        check = model.getAnswer(player,agame,ans)
        
        if check[0] < 0:
            seen =  model.getSeen(player,agame, amoves)
            exclude  = set(seen)
            done = True
            for x in model.keysForGuess(ans):
                if x in exclude:
                    done = False
            
        else:
            #(p,w,l,playerNum, c,y[0],y[1])
            card = check[-1]
            
            amoves.append(check)
            
            player = player + 1
            if player - 1 >= numPlayers:
               player = 1 
    
    return player, agame , amoves


In [127]:
#| hide
s1 = ("simple",False,simpleGuess)
b1 = ("better",False,betterGuess)
d1 =("deep",False,deepGuess)
strategies = [s1, d1,b1,b1,d1,s1]
counts = [0 for x in range(len(strategies))]
for c  in range(5000):
    p, g, m = runGame(strategies)
    counts[p-1] += 1
counts

[568, 1054, 993, 1014, 923, 448]

In [ ]:
#| hide
chooseCard(rooms,cardFull)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()